In [ ]:
import jupyter_black

jupyter_black.load()

import pandas as pd
import nfl_data_py as nfl
from typing import Literal, Tuple
from src.modelling.metrics import success_rate_lambda
from src.visualisation.visualisation import plot_team_scatter

pd.set_option("display.max_columns", None)

In [ ]:
pbp: pd.DataFrame = nfl.import_pbp_data([2023])

In [ ]:
# offense_players - list of id's
id_df = nfl.import_ids()

In [ ]:
pbp = pbp.query("week<=18")

In [ ]:
pbp["offense_players_list"] = pbp["offense_players"].apply(
    lambda x: x.split(";") if type(x) == str else None
)

In [ ]:
pbp["offense_players_list"]

In [ ]:
id_df.sample(10)

In [ ]:
pbp_player = (
    pbp.explode("offense_players_list")
    .dropna(subset=["offense_players_list"])
    .rename(columns={"offense_players_list": "player_id"})
    .query('player_id!=""')
    .query('play_type=="pass" or play_type=="run"')
    .merge(
        id_df[["gsis_id", "position", "name"]],
        how="left",
        left_on="player_id",
        right_on="gsis_id",
    )
    .drop(columns=["gsis_id"])
)

In [ ]:
pbp_wr = pbp_player.query('position=="WR"').copy()
pbp_wr["own_target"] = pbp_wr.apply(
    lambda row: 1 if row["player_id"] == row["receiver_id"] else 0,
    axis=1,
)
pbp_wr["own_rec"] = pbp_wr.apply(
    lambda row: row["complete_pass"] if row["player_id"] == row["receiver_id"] else 0,
    axis=1,
)
pbp_wr["own_rec_yds"] = pbp_wr.apply(
    lambda row: row["receiving_yards"] if row["player_id"] == row["receiver_id"] else 0,
    axis=1,
)

In [ ]:
# 00-0030279 keenan
# 00-0036963 arsb

In [ ]:
arsb = pbp_wr.query("player_id=='00-0036963'")

In [ ]:
[c for c in arsb.columns if "rec" in c]

In [ ]:
arsb.head()

In [ ]:
arsb_yrr = (
    arsb.groupby("player_id")
    .agg(
        {
            "play_id": "count",
            "pass_attempt": "sum",
            "own_target": "sum",
            "own_rec": "sum",
            "own_rec_yds": "sum",
        }
    )
    .rename(
        columns={
            "play_id": "snaps",
            "pass_attempt": "routes",
            "own_target": "tgts",
            "own_rec": "recs",
            "own_rec_yds": "yds",
        }
    )
    .astype(int)
)
arsb_yrr["yrr"] = (arsb_yrr["yds"] / arsb_yrr["routes"]).round(3)
arsb_yrr

In [ ]:
# these are snaps he was on the field for, i also need team totals